In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [2]:
# Define the LEMCell
class LEMCell(nn.Module):
    def __init__(self, ninp, nhid, dt):
        super(LEMCell, self).__init__()
        self.ninp = ninp
        self.nhid = nhid
        self.dt = dt
        self.inp2hid = nn.Linear(ninp, 4 * nhid)
        self.hid2hid = nn.Linear(nhid, 3 * nhid)
        self.transform_z = nn.Linear(nhid, nhid)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.nhid)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, y, z):
        transformed_inp = self.inp2hid(x)
        transformed_hid = self.hid2hid(y)
        i_dt1, i_dt2, i_z, i_y = transformed_inp.chunk(4, 1)
        h_dt1, h_dt2, h_y = transformed_hid.chunk(3, 1)

        ms_dt_bar = self.dt * torch.sigmoid(i_dt1 + h_dt1)
        ms_dt = self.dt * torch.sigmoid(i_dt2 + h_dt2)

        z = (1. - ms_dt) * z + ms_dt * torch.tanh(i_y + h_y)
        y = (1. - ms_dt_bar) * y + ms_dt_bar * torch.tanh(self.transform_z(z) + i_z)

        return y, z

# Define the LEM model
class LEM(nn.Module):
    def __init__(self, ninp, nhid, nout, dt=1.):
        super(LEM, self).__init__()
        self.nhid = nhid
        self.cell = LEMCell(ninp, nhid, dt)
        self.classifier = nn.Linear(nhid, nout)
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'classifier' in name and 'weight' in name:
                nn.init.kaiming_normal_(param.data)

    def forward(self, input):
        y = input.data.new(input.size(1), self.nhid).zero_()
        z = input.data.new(input.size(1), self.nhid).zero_()
        for x in input:
            y, z = self.cell(x, y, z)
        out = self.classifier(y)
        return out


### PINN data importing

In [3]:
# importing data

# Load the .mat file
mat_data = scipy.io.loadmat('burg.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u = mat_data['u1']



### Exact Solution data importing

In [4]:
# importing data

import torch
import torch.nn as nn
import numpy as np
import time
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('burgers_shock.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u_1 = mat_data['usol']


In [5]:
# Set random seed for reproducibility
#torch.manual_seed(42)

# Toy problem data
input_size = 256
hidden_size = 32
output_size = 256
sequence_length = 79
batch_size = 1
num_epochs = 20000

# Set random seed for reproducibility
#torch.manual_seed(42)
u[:, 0:100].shape

(256, 100)

In [6]:
input_data = u[:,0:79]
target_data = u[:,1:80]

test_data = u[:,79]
#test_target = u[:,80:100]

print("test data shape", test_data.shape)
#print("test target shape", test_target.shape)

print("input data shape",input_data.shape)
print("Target data shape",target_data.shape)

# Convert data to tensors
input_tensor = torch.tensor(input_data.T).view(batch_size, sequence_length, input_size).float()
target_tensor = torch.tensor(target_data.T).view(batch_size, sequence_length, output_size).float()

print("input tensor shape",input_tensor.shape)
print("Target tensor shape",target_tensor.shape)

test data shape (256,)
input data shape (256, 79)
Target data shape (256, 79)
input tensor shape torch.Size([1, 79, 256])
Target tensor shape torch.Size([1, 79, 256])


In [7]:
# Convert test data to tensors
test_tensor = torch.tensor(test_data.T).view(batch_size, 1, input_size).float()
#test_target_tensor = torch.tensor(test_target.T).view(batch_size, 20, output_size).float()
target_tensor = torch.squeeze(target_tensor)

In [8]:
# Create LEM instance
lem = LEM(input_size, hidden_size, output_size, dt=0.1)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lem.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    output = lem(input_tensor)
    loss = criterion(output, target_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.16f}')

Epoch: 10/20000, Loss: 0.3883340060710907
Epoch: 20/20000, Loss: 0.3492854237556458
Epoch: 30/20000, Loss: 0.3117892742156982
Epoch: 40/20000, Loss: 0.2760362327098846
Epoch: 50/20000, Loss: 0.2424257993698120
Epoch: 60/20000, Loss: 0.2119646221399307
Epoch: 70/20000, Loss: 0.1848683953285217
Epoch: 80/20000, Loss: 0.1605141758918762
Epoch: 90/20000, Loss: 0.1395493745803833
Epoch: 100/20000, Loss: 0.1214146092534065
Epoch: 110/20000, Loss: 0.1060409545898438
Epoch: 120/20000, Loss: 0.0930523723363876
Epoch: 130/20000, Loss: 0.0819733664393425
Epoch: 140/20000, Loss: 0.0724668055772781
Epoch: 150/20000, Loss: 0.0643650814890862
Epoch: 160/20000, Loss: 0.0575365535914898
Epoch: 170/20000, Loss: 0.0518238022923470
Epoch: 180/20000, Loss: 0.0470631383359432
Epoch: 190/20000, Loss: 0.0431018471717834
Epoch: 200/20000, Loss: 0.0398043431341648
Epoch: 210/20000, Loss: 0.0370527319610119
Epoch: 220/20000, Loss: 0.0347453020513058
Epoch: 230/20000, Loss: 0.0327953919768333
Epoch: 240/20000, Lo

Epoch: 1960/20000, Loss: 0.0004427622479852
Epoch: 1970/20000, Loss: 0.0004371681425255
Epoch: 1980/20000, Loss: 0.0004324385954533
Epoch: 1990/20000, Loss: 0.0004262750153430
Epoch: 2000/20000, Loss: 0.0004204456345178
Epoch: 2010/20000, Loss: 0.0004152093606535
Epoch: 2020/20000, Loss: 0.0004099020734429
Epoch: 2030/20000, Loss: 0.0004047988040838
Epoch: 2040/20000, Loss: 0.0003997650637757
Epoch: 2050/20000, Loss: 0.0003948319063056
Epoch: 2060/20000, Loss: 0.0003899903967977
Epoch: 2070/20000, Loss: 0.0003852364316117
Epoch: 2080/20000, Loss: 0.0003805686719716
Epoch: 2090/20000, Loss: 0.0003759846731555
Epoch: 2100/20000, Loss: 0.0003714827471413
Epoch: 2110/20000, Loss: 0.0003670608566608
Epoch: 2120/20000, Loss: 0.0003627503465395
Epoch: 2130/20000, Loss: 0.0003610793210100
Epoch: 2140/20000, Loss: 0.0003554178983904
Epoch: 2150/20000, Loss: 0.0003502471372485
Epoch: 2160/20000, Loss: 0.0003461833111942
Epoch: 2170/20000, Loss: 0.0003421416622587
Epoch: 2180/20000, Loss: 0.00033

Epoch: 3860/20000, Loss: 0.0000698692965670
Epoch: 3870/20000, Loss: 0.0000693635520292
Epoch: 3880/20000, Loss: 0.0000688623840688
Epoch: 3890/20000, Loss: 0.0000683665130055
Epoch: 3900/20000, Loss: 0.0000679798831698
Epoch: 3910/20000, Loss: 0.0000797752290964
Epoch: 3920/20000, Loss: 0.0000671224843245
Epoch: 3930/20000, Loss: 0.0000664447870804
Epoch: 3940/20000, Loss: 0.0000659741272102
Epoch: 3950/20000, Loss: 0.0000655134790577
Epoch: 3960/20000, Loss: 0.0000650598958600
Epoch: 3970/20000, Loss: 0.0000646028493065
Epoch: 3980/20000, Loss: 0.0000641385122435
Epoch: 3990/20000, Loss: 0.0000636796321487
Epoch: 4000/20000, Loss: 0.0000632327501080
Epoch: 4010/20000, Loss: 0.0000627890258329
Epoch: 4020/20000, Loss: 0.0000623492887826
Epoch: 4030/20000, Loss: 0.0000619131460553
Epoch: 4040/20000, Loss: 0.0000614806849626
Epoch: 4050/20000, Loss: 0.0000610517454334
Epoch: 4060/20000, Loss: 0.0000606263056397
Epoch: 4070/20000, Loss: 0.0000602045329288
Epoch: 4080/20000, Loss: 0.00005

Epoch: 5800/20000, Loss: 0.0000208324527193
Epoch: 5810/20000, Loss: 0.0000207028442674
Epoch: 5820/20000, Loss: 0.0000205852757063
Epoch: 5830/20000, Loss: 0.0000204734224099
Epoch: 5840/20000, Loss: 0.0000203605886782
Epoch: 5850/20000, Loss: 0.0000202493592951
Epoch: 5860/20000, Loss: 0.0000201386410481
Epoch: 5870/20000, Loss: 0.0000200287249754
Epoch: 5880/20000, Loss: 0.0000199194801098
Epoch: 5890/20000, Loss: 0.0000198109028133
Epoch: 5900/20000, Loss: 0.0000197030076379
Epoch: 5910/20000, Loss: 0.0000195957691176
Epoch: 5920/20000, Loss: 0.0000194891817955
Epoch: 5930/20000, Loss: 0.0000193832474906
Epoch: 5940/20000, Loss: 0.0000192779425561
Epoch: 5950/20000, Loss: 0.0000191733051906
Epoch: 5960/20000, Loss: 0.0000190693972399
Epoch: 5970/20000, Loss: 0.0000189809907170
Epoch: 5980/20000, Loss: 0.0000228392855206
Epoch: 5990/20000, Loss: 0.0000264718855760
Epoch: 6000/20000, Loss: 0.0000209455611184
Epoch: 6010/20000, Loss: 0.0000195676657313
Epoch: 6020/20000, Loss: 0.00001

Epoch: 7700/20000, Loss: 0.0000082287097030
Epoch: 7710/20000, Loss: 0.0000081918760770
Epoch: 7720/20000, Loss: 0.0000081552016127
Epoch: 7730/20000, Loss: 0.0000081186790339
Epoch: 7740/20000, Loss: 0.0000080823119788
Epoch: 7750/20000, Loss: 0.0000080460931713
Epoch: 7760/20000, Loss: 0.0000080100271589
Epoch: 7770/20000, Loss: 0.0000079741503214
Epoch: 7780/20000, Loss: 0.0000079437668319
Epoch: 7790/20000, Loss: 0.0000094104225354
Epoch: 7800/20000, Loss: 0.0000083062859630
Epoch: 7810/20000, Loss: 0.0000078762122939
Epoch: 7820/20000, Loss: 0.0000080778627307
Epoch: 7830/20000, Loss: 0.0000080353702288
Epoch: 7840/20000, Loss: 0.0000079027431639
Epoch: 7850/20000, Loss: 0.0000077747990872
Epoch: 7860/20000, Loss: 0.0000077034183050
Epoch: 7870/20000, Loss: 0.0000076536380220
Epoch: 7880/20000, Loss: 0.0000076151409303
Epoch: 7890/20000, Loss: 0.0000075811667557
Epoch: 7900/20000, Loss: 0.0000075476577877
Epoch: 7910/20000, Loss: 0.0000075137609201
Epoch: 7920/20000, Loss: 0.00000

...

Epoch: 11600/20000, Loss: 0.0000013769392808
Epoch: 11610/20000, Loss: 0.0000013715340401
Epoch: 11620/20000, Loss: 0.0000013665079450
Epoch: 11630/20000, Loss: 0.0000013638857581
Epoch: 11640/20000, Loss: 0.0000017205672975
Epoch: 11650/20000, Loss: 0.0000201526163437
Epoch: 11660/20000, Loss: 0.0000032735274544
Epoch: 11670/20000, Loss: 0.0000018112496036
Epoch: 11680/20000, Loss: 0.0000015146664509
Epoch: 11690/20000, Loss: 0.0000013933675973
Epoch: 11700/20000, Loss: 0.0000013514992361
Epoch: 11710/20000, Loss: 0.0000013380492874
Epoch: 11720/20000, Loss: 0.0000013296684074
Epoch: 11730/20000, Loss: 0.0000013212879821
Epoch: 11740/20000, Loss: 0.0000013142381476
Epoch: 11750/20000, Loss: 0.0000013083864587
Epoch: 11760/20000, Loss: 0.0000013032204151
Epoch: 11770/20000, Loss: 0.0000012980407291
Epoch: 11780/20000, Loss: 0.0000012930266848
Epoch: 11790/20000, Loss: 0.0000012880261693
Epoch: 11800/20000, Loss: 0.0000012830853393
Epoch: 11810/20000, Loss: 0.0000012781723626
Epoch: 118

Epoch: 13470/20000, Loss: 0.0000007573898984
Epoch: 13480/20000, Loss: 0.0000007456082471
Epoch: 13490/20000, Loss: 0.0000007434902045
Epoch: 13500/20000, Loss: 0.0000007405440101
Epoch: 13510/20000, Loss: 0.0000007373478752
Epoch: 13520/20000, Loss: 0.0000007348589861
Epoch: 13530/20000, Loss: 0.0000007322515216
Epoch: 13540/20000, Loss: 0.0000007297457500
Epoch: 13550/20000, Loss: 0.0000007272713560
Epoch: 13560/20000, Loss: 0.0000007248164593
Epoch: 13570/20000, Loss: 0.0000007223773082
Epoch: 13580/20000, Loss: 0.0000007199579954
Epoch: 13590/20000, Loss: 0.0000007175549399
Epoch: 13600/20000, Loss: 0.0000007154048376
Epoch: 13610/20000, Loss: 0.0000007337789043
Epoch: 13620/20000, Loss: 0.0000027670130294
Epoch: 13630/20000, Loss: 0.0000025678916700
Epoch: 13640/20000, Loss: 0.0000010178965795
Epoch: 13650/20000, Loss: 0.0000009516553519
Epoch: 13660/20000, Loss: 0.0000007347942983
Epoch: 13670/20000, Loss: 0.0000007114250593
Epoch: 13680/20000, Loss: 0.0000007037343721
Epoch: 136

Epoch: 15430/20000, Loss: 0.0000004539749909
Epoch: 15440/20000, Loss: 0.0000004528941133
Epoch: 15450/20000, Loss: 0.0000004518475691
Epoch: 15460/20000, Loss: 0.0000004524774226
Epoch: 15470/20000, Loss: 0.0000006670468338
Epoch: 15480/20000, Loss: 0.0000175294408109
Epoch: 15490/20000, Loss: 0.0000008359942854
Epoch: 15500/20000, Loss: 0.0000005728721248
Epoch: 15510/20000, Loss: 0.0000005207870117
Epoch: 15520/20000, Loss: 0.0000005072410545
Epoch: 15530/20000, Loss: 0.0000004989119020
Epoch: 15540/20000, Loss: 0.0000004766588120
Epoch: 15550/20000, Loss: 0.0000004533236222
Epoch: 15560/20000, Loss: 0.0000004463830976
Epoch: 15570/20000, Loss: 0.0000004462796426
Epoch: 15580/20000, Loss: 0.0000004438300891
Epoch: 15590/20000, Loss: 0.0000004427530200
Epoch: 15600/20000, Loss: 0.0000004414355317
Epoch: 15610/20000, Loss: 0.0000004403407274
Epoch: 15620/20000, Loss: 0.0000004392523181
Epoch: 15630/20000, Loss: 0.0000004381755332
Epoch: 15640/20000, Loss: 0.0000004371222531
Epoch: 156

Epoch: 17320/20000, Loss: 0.0000003346021344
Epoch: 17330/20000, Loss: 0.0000003339923467
Epoch: 17340/20000, Loss: 0.0000003333830136
Epoch: 17350/20000, Loss: 0.0000003327815534
Epoch: 17360/20000, Loss: 0.0000003321826796
Epoch: 17370/20000, Loss: 0.0000003315873869
Epoch: 17380/20000, Loss: 0.0000003310097156
Epoch: 17390/20000, Loss: 0.0000003319989617
Epoch: 17400/20000, Loss: 0.0000007053773743
Epoch: 17410/20000, Loss: 0.0000164771190612
Epoch: 17420/20000, Loss: 0.0000059495978348
Epoch: 17430/20000, Loss: 0.0000017065904103
Epoch: 17440/20000, Loss: 0.0000004653302881
Epoch: 17450/20000, Loss: 0.0000003384101035
Epoch: 17460/20000, Loss: 0.0000003479215991
Epoch: 17470/20000, Loss: 0.0000003426916635
Epoch: 17480/20000, Loss: 0.0000003355082754
Epoch: 17490/20000, Loss: 0.0000003315690265
Epoch: 17500/20000, Loss: 0.0000003299271327
Epoch: 17510/20000, Loss: 0.0000003291566202
Epoch: 17520/20000, Loss: 0.0000003284181105
Epoch: 17530/20000, Loss: 0.0000003275553127
Epoch: 175

Epoch: 19150/20000, Loss: 0.0000002709690534
Epoch: 19160/20000, Loss: 0.0000002681487672
Epoch: 19170/20000, Loss: 0.0000002674422035
Epoch: 19180/20000, Loss: 0.0000002670088577
Epoch: 19190/20000, Loss: 0.0000002665093461
Epoch: 19200/20000, Loss: 0.0000002658512983
Epoch: 19210/20000, Loss: 0.0000002654477669
Epoch: 19220/20000, Loss: 0.0000002650240276
Epoch: 19230/20000, Loss: 0.0000002651368618
Epoch: 19240/20000, Loss: 0.0000002818022722
Epoch: 19250/20000, Loss: 0.0000014423060293
Epoch: 19260/20000, Loss: 0.0000026766690553
Epoch: 19270/20000, Loss: 0.0000020856584797
Epoch: 19280/20000, Loss: 0.0000002832030077
Epoch: 19290/20000, Loss: 0.0000005555041298
Epoch: 19300/20000, Loss: 0.0000002952670570
Epoch: 19310/20000, Loss: 0.0000002952520788
Epoch: 19320/20000, Loss: 0.0000002659535312
Epoch: 19330/20000, Loss: 0.0000002691279519
Epoch: 19340/20000, Loss: 0.0000002640538810
Epoch: 19350/20000, Loss: 0.0000002623800128
Epoch: 19360/20000, Loss: 0.0000002617937582
Epoch: 193

In [9]:
print(test_tensor.shape)
prediction_tensor = torch.zeros(1, 20, 256).float()
print(prediction_tensor.shape)

torch.Size([1, 1, 256])
torch.Size([1, 20, 256])


In [10]:
with torch.no_grad():
    prediction = lem(test_tensor)
    prediction = prediction.view(1, 1, 256).float()
    prediction_tensor[:, 0, :] = prediction
    for i in range(19):
        prediction = lem(prediction)
        prediction = prediction.view(1, 1, 256).float()
        prediction_tensor[:, i+1, :] = prediction
        

### Four different types of error

In [11]:
# Exact Solution

u_test = u_1.T
u_test_full = u_test[80:100, :]

In [12]:
prediction_tensor = torch.squeeze(prediction_tensor)


In [13]:
# Extrapolation

k1 = ( prediction_tensor - u_test_full)**2
u_test_full_tensor = torch.tensor(u_test_full**2)
u_test_full_tensor.shape

torch.Size([20, 256])

### L^2 norm error

In [14]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(k1)/ torch.mean(u_test_full_tensor)

print("Relative Error Test: ", relative_error_test.item(), "%")

Relative Error Test:  0.043790851178600466 %


In [15]:
2+

SyntaxError: invalid syntax (4209523232.py, line 1)

### Max absolute norm error

In [ ]:
R_abs = torch.max(torch.abs(prediction_tensor - u_test_full))

In [ ]:
print(R_abs)

### Explained variance score

In [ ]:
import torch

a = prediction_tensor
b = u_test_full
# Assuming 'a' is your predicted values (model's predictions) and 'b' is the true values (ground truth)
# Make sure 'a' and 'b' are PyTorch tensors
# a = torch.tensor(a)
b = torch.tensor(b)
# Calculate the mean of 'b'
mean_b = torch.mean(b)

# Calculate the Explained Variance Score
numerator = torch.var(b - a)  # Variance of the differences between 'b' and 'a'
denominator = torch.var(b)    # Variance of 'b'
evs = 1 - numerator / denominator

print("Explained Variance Score:", evs.item())

### Mean absolute error

In [ ]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(torch.abs(prediction_tensor - u_test_full))

print("Relative Error Test: ", relative_error_test, "%")

### Contour plot for PINN (80 percent) and (20 percentage lem prediction)

In [ ]:
print(prediction_tensor.shape)
prediction_tensor = torch.squeeze(prediction_tensor)
input_tensor = torch.squeeze(input_tensor)

conc_u = torch.squeeze(input_tensor)
concatenated_tensor = torch.cat((conc_u, prediction_tensor), dim=0)

x1 = np.linspace(-1, 1, 256)
t1 = np.linspace(0, 1, 99)

### Snapshot time plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed




final_time_output = prediction_tensor[3, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, 83].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.83}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.83_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.83_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed



final_time_output = prediction_tensor[-2, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, -2].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.98}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.98_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.98_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


### Contour plot where 80 percent for PINN solution and 20 percent for lem solution

### Exact contour

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = u_1.T

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

#plt.savefig('Contour_Exact.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_exact.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = concatenated_tensor.numpy()

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t1)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

# Add a dotted line at t = 0.8
plt.axvline(x=0.8, color='black', linestyle='dotted', linewidth=5)

#plt.savefig('Contour_LEM_20.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_LEM_20.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()
